In [ ]:
import config
from model.cnn_document_model import DocumentModel
from preprocessing.utils import Preprocess, remove_empty_docs
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize                                     

import keras.backend as K

## Load pre-trained IMDB model and data

In [ ]:
imdb_model = DocumentModel.load_model(config.MODEL_DIR+ '/imdb/model_02.json')
imdb_model.load_model_weights(config.MODEL_DIR+ '/imdb/model_02.hdf5')

model = imdb_model.get_classification_model()
model.compile(loss="binary_crossentropy", optimizer='rmsprop', metrics=["accuracy"])

In [ ]:
train_df = Loader.load_imdb_data(directory = 'train')
print(train_df.shape)

corpus = train_df['review'].tolist()
target = train_df['sentiment'].tolist()
corpus, target = remove_empty_docs(corpus, target)
print(len(corpus))


## Pre process input and compute document embeddings

In [ ]:
preprocessor = Preprocess(corpus=corpus)
corpus_to_seq = preprocessor.fit()

corpus = train_df['review'].tolist()
target = train_df['sentiment'].tolist()
corpus_to_seq = preprocessor.transform(corpus)

x_train = np.array(corpus_to_seq)
y_train = np.array(target)

print(x_train.shape, y_train.shape)

In [ ]:
print('Evaluating Model ...')
print(model.evaluate(x_train, y_train))

preds = model.predict(x_train)

#invert predicted label
pseudo_label = np.subtract(1,preds)

## Gradient Calculation of inverted output w.r.t sentence embeddings

In [ ]:
#Get the learned sentence embeddings
sentence_ebd = imdb_model.get_sentence_model().predict(x_train)

input_tensors = [model.inputs[0], # input data
                 model.sample_weights[0], # how much to weight each sample by
                 model.targets[0], # labels                 
]
#variable tensor at the sentence embeding layer
weights = imdb_model.get_sentence_model().outputs

#calculate gradient of the total model loss w.r.t 
#the variables at sentence embd layer                                     
gradients = model.optimizer.get_gradients(model.total_loss, weights) 
get_gradients = K.function(inputs=input_tensors, outputs=gradients)

In [ ]:
document_number = 2
K.set_learning_phase(0)
inputs = [[x_train[document_number]], # X
          [1], # sample weights
          [[pseudo_label[document_number][0]]], # y
]
grad = get_gradients(inputs)

sent_score = []
for i in range(Preprocess.NUM_SENTENCES):
    #sent_score.append((i, -np.abs(np.dot(grad[0][0][i],sentence_ebd[document_number][i])))) #DECREASING
    sent_score.append((i, -np.linalg.norm(grad[0][0][i])))

sent_score.sort(key=lambda tup: tup[1])
summary_sentences = [ i for i, s in sent_score[:4]]

doc = corpus[document_number]
label = y_train[document_number]
prediction = preds[document_number]
print(doc, label , prediction)

sentences = sent_tokenize(doc)
for i in summary_sentences:
    print(i, sentences[i])
    
